# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>1 |</span></b> <b>INTRODUCTION</b></div>

### <b><span style='color:#A51C30'> 1.1 </span> GPU Dtype</b> 

- **`P100 GPU`**: For High Performance Computing(HPC)
- **`T4 GPU`**: For Deep Learning and AI tasks

### <b><span style='color:#A51C30'> 1.2 </span> EEG Band</b>

<div style="border-radius:10px; border: #babab5 solid; padding: 15px; background-color:##A51C30; font-size:100%;">

📌 **Check out**
 
 - Delta, Belta is most evident in frontally 
 - Alpha is both side & posterior region and then c3 & c4 at rest 
 - Delta, Belta, Alpha, Theta Frequency is in 0~30Hz

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F16438831%2Ffc9b51893c3fcb785de42e2161357ac4%2Feeg%20band.PNG?generation=1708142412775439&alt=media)

### <b><span style='color:#A51C30'> 1.3 </span> New Chain</b> 

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F16438831%2Fee75fb2b76b12fda5c559504eacc09b8%2FCentral.PNG?generation=1708318428173795&alt=media)

### <b><span style='color:#A51C30'> 1.4 </span> Updated!</b> 

<div style="border-radius:10px; border: #babab5 solid; padding: 15px; background-color:##A51C30; font-size:100%;">
    
📌 **`VER1`**: Original Chain, 0~30Hz, [4Waveblcok + 1GRU] 
    
📌 **`VER2`**: Adding Central[Cz,Fz,C4,C3], 0~30Hz, 


# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>2 |</span></b> <b>GPUs Setting</b></div>

In [ ]:
VER = 2

import os
import gc
import ctypes

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
print('tensorflow version:',tf.__version__)

# CUDA 0,1
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# gpu strategy
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) <= 1:
    strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')
    print(f'Using {len(gpus)} gpus')
else: 
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} gpus')
    
# warning filtering
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# mixed_preicision
# Helps memeory effectively 

MIX=True

if MIX: 
    tf.config.optimizer.set_experimental_options({'auto_mixed_precision':True})
    print("Mixed Precision Enabled")
else:
    print("Using Full Precision")

**Clean Memory**

In [ ]:
def clean_memory():
    # malloc_trim: 현재 사용되지 않는 메모리를 시스템에서 다시 반환함0
    ctypes.CDLL('libc.so.6').malloc_trim(0)
    gc.collect()
clean_memory()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>3 |</span></b> <b>Load Train Data</b></div>

In [ ]:
train = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/train.csv")
print('train shape: ',train.shape)
display(train.head())

### <b><span style='color:#A51C30'> 3.1 </span> Raw EEG Signals</b>

In [ ]:
New_Chain = True

In [ ]:
df = pd.read_parquet('/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1000913311.parquet')
FEATS = df.columns
print(f'There are {len(FEATS)} raw eeg features')
print(list(FEATS))

In [ ]:
if New_Chain: 
    print('We will use the follwing subset of 10 raw eeg features:')
    FEATS = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2','Fz', 'Pz']
    FEAT2IDX = {x:y for x,y in zip(FEATS, range(len(FEATS)))}
    print(FEATS)

In [ ]:
def eeg_from_parquet(parquet_path, display=False):
    
    # 특정 electrode만 추출하기
    eeg = pd.read_parquet(parquet_path, columns=FEATS)
    rows = len(eeg)
    offset = (rows-10000)//2
    # eeg의 구간: 50초
    eeg = eeg.iloc[offset:offset+10_000]
    
    if display:
        plt.figure(figsize=(10,5))
        offset = 0
        
    # CONVERT TO NUMPY
    data = np.zeros((10_000, len(FEATS)))
    for j,col in enumerate(FEATS):
        
        # FILL NAN
        x = eeg[col].values.astype('float32')
        m = np.nanmean(x)
        if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
        else: x[:] = 0
            
        data[:,j] = x    
        
        if display: 
            if j!=0: offset += x.max()
            plt.plot(range(10_000), x-offset, label=col)
            offset -= x.min()
            
    if display:
        plt.legend()
        name = parquet_path.split('/')[-1]
        name = name.split('.')[0]
        plt.title(f'EEG {name}', size=16)
        plt.show()
        
    return data    

In [ ]:
%%time

CREATE_EEGS = False
all_eegs = {}
DISPLAY = 4
EEG_IDS = train.eeg_id.unique()
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'

for i, eeg_id in enumerate(EEG_IDS):
    if (i%100==0)&(i!=0): print(i, ', ', end='')
        
    data = eeg_from_parquet(f'{PATH}{eeg_id}.parquet', display=i<DISPLAY) 
    all_eegs[eeg_id] = data
    
    if i==DISPLAY:
        if CREATE_EEGS:
            print(f'processing {train.eeg_id.nunique()} eeg parquets... ', end='')
        else:
            print(f'Reading {len(EEG_IDS)} eeg Numpys from disk.')
            
            break

if New_Chain:   
    all_eegs = np.load('/kaggle/input/central-part/eegs.npy',allow_pickle=True).item()  

### <b><span style='color:#A51C30'> 3.2 </span> Deduplicate Train EEG Id</b>

In [ ]:
# LOAD TRAIN 
df = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df.columns[-6:]
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

train = df.groupby('eeg_id')[['patient_id']].agg('first')

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
train = train.loc[train.eeg_id.isin(EEG_IDS)]
print('Train Data with unique eeg_id shape:', train.shape )
train.head()

**Expert_Agreement**

In [ ]:
def expert_agreement(row):
    max_vote = row[TARGETS].max()
    if max_vote == 1:
        return 'Idealized'
    elif (max_vote < 1) & (max_vote >= 0.85):
        return 'Well'
    elif (max_vote <0.85) & (max_vote >= 0.70):
        return 'Great'
    elif (max_vote <0.70) & (max_vote >= 0.55):
        return 'Good'
    elif (max_vote <0.55) & (max_vote >= 0.40):
        return 'NotBad'
    else:
        return 'Undecided'
        
train['pattern'] = train.apply(expert_agreement, axis=1)    

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(data=train, x='pattern')
plt.xlabel("expert agreement")
plt.ylabel("Count")
plt.legend()
plt.title("Distribution of expert agreeement")

### <b><span style='color:#A51C30'> 3.3 </span> Butter Low-Pass Filter[0~30Hz]</b>

In [ ]:
from scipy.signal import butter, lfilter

def butter_lowpass_filter(data, cutoff_freq=30, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

### <b><span style='color:#A51C30'> 3.4 </span> Wavelet Denoising[db8]</b>

In [ ]:
import pywt
print("The wavelet functions we can use:")
print(pywt.wavelist())

USE_WAVELET = 'db8'

In [ ]:
# DENOISE FUNCTION
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def denoise(x, wavelet='haar', level=1):    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    ret=pywt.waverec(coeff, wavelet, mode='per')
    
    return ret

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>4 |</span></b> <b>Data Loader</b></div>

In [ ]:
import tensorflow as tf

class DataGenerator(tf.keras.utils.Sequence):
    # Constructor(init method)
    def __init__(self,data, batch_size=32, shuffle=True, eegs=all_eegs,
               mode='train', downsample=5,):
        
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.eegs = eegs 
        self.mode = mode
        self.downsample = downsample
        self.on_epoch_end()
        
    def __len__(self):
        ct = int(np.ceil(len(self.data)/self.batch_size))
        return ct
        
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        return X[:,::self.downsample,:], y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle: np.random.shuffle(self.indexes)
        
    def __data_generation(self, indexes):
        # New chain change 8 -> 10
        X = np.zeros((len(indexes),10_000,10), dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')
        
        sample = np.zeros((10_000, X.shape[-1]))
        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            data = self.eegs[row.eeg_id] 
            
            # FEATURE ENGINEER
            sample[:,0] = data[:,FEAT2IDX['Fp1']] - data[:,FEAT2IDX['T3']]
            sample[:,1] = data[:,FEAT2IDX['T3']] - data[:,FEAT2IDX['O1']]
            
            sample[:,2] = data[:,FEAT2IDX['Fp1']] - data[:,FEAT2IDX['C3']]
            sample[:,3] = data[:,FEAT2IDX['C3']] - data[:,FEAT2IDX['O1']]
            
            sample[:,4] = data[:,FEAT2IDX['Fp2']] - data[:,FEAT2IDX['C4']]
            sample[:,5] = data[:,FEAT2IDX['C4']] - data[:,FEAT2IDX['O2']]
            
            sample[:,6] = data[:,FEAT2IDX['Fp2']] - data[:,FEAT2IDX['T4']]
            sample[:,7] = data[:,FEAT2IDX['T4']] - data[:,FEAT2IDX['O2']]
            
            sample[:,8] = data[:,FEAT2IDX['Fz']] - data[:,FEAT2IDX['Pz']]
            sample[:,9] = data[:,FEAT2IDX['C3']] - data[:,FEAT2IDX['C4']]
            
            # crop scaling
            sample = np.clip(sample, -1024,1024)

            
            # standarization: (x-mean)/std
            # The mean of all the raw data: 0
            # The std of all the raw data: 32
            sample = np.nan_to_num(sample,nan=0) / 32.0
            
            # BUTTER LOW-PASS FILTER[0~30Hz]
            sample = butter_lowpass_filter(sample)
            
            # Wavelet Filtering[db6]
            sample = denoise(sample,wavelet=USE_WAVELET)
        
            X[j,] = sample
            if self.mode!= 'test':
                y[j] = row[TARGETS]
                
        return X,y
    

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>5 |</span></b> <b>Build WaveNet Model</b></div>

In [ ]:
# TRAIN SCHEDULE

def lrfn(epoch):
    return [1e-3,1e-3,1e-4,1e-4,1e-5][epoch]
LR = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
EPOCHS = 5

### <b><span style='color:#A51C30'> 5.1 </span> Wave_Block </b>

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F16438831%2F57a7deab91934b33bf8bc3c334a16512%2Fw.PNG?generation=1708167117378209&alt=media)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Conv1D, Concatenate,AveragePooling1D, Bidirectional, LSTM


def wave_block(x, filters, kernel_size, n):
    # dilation_rates: Convultion Layer 매개변수 중 하나
    # Convultion filter의 간격을 조절하는 역할
    dilation_rates = [2**i for i in range(n)]
    # Convultion 1D Layer
    x = Conv1D(filters = filters,
              kernel_size = 1,
              padding = 'same')(x)
    res_x = x
    # Activation Function: tanh, sigm
    for dilation_rate in dilation_rates:
        tanh_out = Conv1D(filters = filters, 
                         kernel_size = kernel_size,
                         padding = 'same',
                         activation = 'tanh',
                         dilation_rate = dilation_rate)(x)
        
        sigm_out = Conv1D(filters = filters,
                         kernel_size = kernel_size,
                         padding = 'same',
                         activation = 'sigmoid',
                         dilation_rate = dilation_rate)(x)
        # Multiply: 두 텐서의 요소를 곱하는 Layer
        x = Multiply()([tanh_out, sigm_out])
        x = Conv1D(filters=filters,
                  kernel_size = 1,
                  padding = 'same')(x)
        # Add: 두 텐서의 요소를 더하는 layer 
        res_x = Add()([res_x, x])
        
    return res_x    
        

In [ ]:
def positional_encoding(maxlen, num_hid):
        depth = num_hid/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = tf.concat(
          [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
          axis=-1)
        return pos_encoding

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, feat_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="gelu"), layers.Dense(feat_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

### <b><span style='color:#A51C30'> 5.2 </span> Wave Net + RNN Architecture </b>

In [ ]:
embed_dim = 32  
num_heads = 4  
ff_dim = 128  
dropout_rate = 0.0


def build_model():
    
    # INPUT
    inp = tf.keras.Input(shape=(2_000,10))
    
    # Waveblock
    inp2 = tf.keras.Input(shape=(2_000,1))
    x = layers.Dense(16)(inp2)
    p = positional_encoding(2000,16)
    x = x + p
    
    x = wave_block(inp2, 16, 3, 12)
    x = wave_block(x, 16, 5, 12)
    x = tf.keras.layers.BatchNormalization()(x)
    x = AveragePooling1D(pool_size=2)(x)
    
    x = wave_block(x, 32, 3, 8)
    x = wave_block(x, 32, 5, 8)
    x = tf.keras.layers.BatchNormalization()(x)
    x = AveragePooling1D(pool_size=2)(x)
    
    x = wave_block(x, 64, 3, 4)
    x = wave_block(x, 64, 5, 4)
    x = tf.keras.layers.BatchNormalization()(x)
    x = AveragePooling1D(pool_size=2)(x)
    x = TransformerBlock(embed_dim, 64, num_heads, ff_dim, dropout_rate)(x)


    model2 = tf.keras.Model(inputs=inp2, outputs=x)
    
    #####
    
    # LEFT TEMPORAL CHAIN
    x1 = model2(inp[:,:,0:1])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:,:,1:2])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z1 = tf.keras.layers.Average()([x1,x2])
    
    # LEFT PARASAGITTAL CHAIN
    x1 = model2(inp[:,:,2:3])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:,:,3:4])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z2 = tf.keras.layers.Average()([x1,x2])
    
    # RIGHT PARASAGITTAL CHAIN
    x1 = model2(inp[:,:,4:5])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:,:,5:6])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z3 = tf.keras.layers.Average()([x1,x2])
    
    # RIGHT TEMPORAL CHAIN
    x1 = model2(inp[:,:,6:7])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:,:,7:8])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z4 = tf.keras.layers.Average()([x1,x2])
    
    # Central Chain
    x1 = model2(inp[:,:,8:9])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:,:,9:10])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z5 = tf.keras.layers.Average()([x1,x2])
    
    # COMBINE CHAINS
    y = tf.keras.layers.Concatenate()([z1,z2,z3,z4,z5])
    y = tf.keras.layers.Dense(64, activation='relu')(y)
    y = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(y)
    
    # COMPLIE MODEL 
    model = tf.keras.Model(inputs=inp, outputs=y)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()
    model.compile(loss=loss, optimizer = opt)
    
    return model

In [ ]:
from keras.utils import plot_model
model = build_model() 

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>6 |</span></b> <b>Cross Validation</b></div>

### <b><span style='color:#A51C30'> 6.1 </span> Train GroupKFold</b>

In [ ]:
directory_path = 'WaveNet_Model/'
if not os.path.exists(directory_path):
       os.makedirs(directory_path)

TRAIN_MODEL = True      
FOLDS_TO_TRAIN = 5

from sklearn.model_selection import GroupKFold
import tensorflow.keras.backend as K, gc 

all_oof = [] ; all_true = [] 

gkf = GroupKFold(n_splits=5)
for i, (train_idx, valid_idx) in enumerate(gkf.split(train, train.target, train.patient_id)):
    
    # 진행상태 확인하기 
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_idx)}, valid size {len(valid_idx)}')
    print('#'*25)  
          
    # split train & valid       
    train_gen = DataGenerator(train.iloc[train_idx], shuffle=True, batch_size=32)
    valid_gen = DataGenerator(train.iloc[valid_idx], shuffle=False, batch_size=64)
    
    # model 정의
    # WaveNet Model은 이미 function으로 정함
    K.clear_session()
    with strategy.scope():
        model = build_model()     
          
    # moodel fit
    if TRAIN_MODEL:      
        model.fit(train_gen, verbose=1,
               validation_data = valid_gen, 
              epochs=EPOCHS, callbacks = [LR])  
    # model save
    # save_weights는 tf.keras에서 지원하는 기능
        model.save_weights(f'{directory_path}WaveNet_fold{i}.h5')
                           
    # model predict
    oof = model.predict(valid_gen, verbose=False) 
    all_oof.append(oof)
    all_true.append(train.iloc[valid_idx][TARGETS].values)
                           
    if i == FOLDS_TO_TRAIN-1: break     
                           
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

In [ ]:
del oof, train_gen, valid_gen
clean_memory()

### <b><span style='color:#A51C30'> 6.2 </span> CV Score</b>

In [ ]:
import sys
sys.path.append('/kaggle/input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score with WaveNet Raw EEG =',cv)

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#A51C30"><b><span style='color:#FFFFFF'>7 |</span></b> <b>Submit to Kaggle LB</b></div>

In [ ]:
del all_eegs, train; gc.collect()
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
print('Test shape:',test.shape)
test.head()

In [ ]:
all_eegs2 = {}
DISPLAY = 1
EEG_IDS2 = test.eeg_id.unique()
PATH2 = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/'

print('Processing Test EEG parquets...'); print()
for i,eeg_id in enumerate(EEG_IDS2):
        
    # SAVE EEG TO PYTHON DICTIONARY OF NUMPY ARRAYS
    data = eeg_from_parquet(f'{PATH2}{eeg_id}.parquet', i<DISPLAY)
    all_eegs2[eeg_id] = data

In [ ]:
# INFER MLP ON TEST
preds = []
model = build_model()
test_gen = DataGenerator(test, shuffle=False, batch_size=64, eegs=all_eegs2, mode='test')

print('Inferring test... ',end='')
for i in range(FOLDS_TO_TRAIN):
    print(f'fold {i+1}, ',end='')
    if TRAIN_MODEL:
        model.load_weights(f'WaveNet_Model/WaveNet_fold{i}.h5')
    else:
        model.load_weights(f'/kaggle/input/brain-eegs/WaveNet_Model/WaveNet_fold{i}.h5')
    pred = model.predict(test_gen, verbose=0)
    preds.append(pred)
pred = np.mean(preds,axis=0)
print()
print('Test preds shape',pred.shape)

In [ ]:
# CREATE SUBMISSION.CSV
from IPython.display import display

sub = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub[TARGETS] = pred
sub.to_csv('submission.csv',index=False)
print('Submission shape',sub.shape)
display( sub.head() )

# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
print('Sub row 0 sums to:',sub.iloc[0,-6:].sum())